# **Tell Me What You Want, I'll Tell You What You Like**

In this file, we try to use an approach using Neural Network 

## DOcplex Python API installation

First, we install the docplex python API which we will use for modelling our problem and solving it. (Then, we will not have problem size error)

In [ ]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex -q
        print("!pip install docplex -q...")
    else:
        !pip install --user docplex -q
        print("!pip install --user docplex -q...")
        
print("End of the intallation of python API...")

## Main code

In [ ]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np

import time

In [ ]:
#############################
### Essentials functions ###
#############################

import Function_NN as nn
sys.path.append("..")
import FunctionMain as fm
from Solver import *
from User import *

In [ ]:
# ----------------- Parameters

# The file to consider
file = '../file_with_optimal_val/la04.txt'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = False

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 500

# Number of iteration stop criterion
it_max = 20

# Parameters of the first neural network
nb_hidden_layers = 1
nb_neurons = [1, 1]

# Parameters of the second neural network
nb_hidden_layers2 = 1
nb_neurons2 = [3, 1]


# The number of solutions that we will have in the solver after the first iteration
k_k = 10

In [ ]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver, tasks = fm.initialize_solver(data, n, m, duration)

model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)

# ------------ Solve the model
print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)

# ------------ Display the result
# fm.display_solution(msol, display_sol)
print("Model solved !")

# ---------------- Interaction with the user
print("\n--------Interaction with the user...---------")
print("\nCreating the user...")
user = User()
print("User created !")

#Get the variables of the model

list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# Vector of the start time of each task of each preference
starts = user.start_pref(n, m, display_start)

# Matrix of the start time of each task of each preference
matrix = user.matrix_pref(n, m, display_matrix)

# Testing the order of preferences and the differences between solutions
fm.test(n, m, user)

print("list layers : ",list_layers)

sol_layers = fm.list_list_list_start_of_tasks(n, m, list_layers)

In [ ]:
from sklearn.model_selection import train_test_split

X_list_start_sol = []
Y_list_layers = []
for i in range(len(list_layers)):
    for j in range(len(list_layers[i])):
        list_start = fm.start_sol(n, m, list_layers[i][j])
        # X : Liste de solutions par layer (List de list de starts de chaque solution)
        X_list_start_sol.append(list_start)
        # Y : Liste des layers de chaque solution (List de int)
        if i == 0:
            Y_list_layers.append(1)
        else:
            Y_list_layers.append(0)
# nb_sols = len(X_list_start_sol)
X_train, X_test, y_train, y_test = train_test_split(X_list_start_sol, Y_list_layers, test_size=0.2, random_state=None)


In [77]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

# Définition de la fonction d'activation personnalisée
def custom_activation(x):
    threshold = 0.5
    return K.cast(K.greater(x, threshold), dtype='float32')

# Créer un modèle séquentiel (une pile linéaire de couches)
model = Sequential()

# Ajouter une couche cachée avec 16 neurones et une fonction d'activation ReLU
model.add(Dense(2, input_dim=n*m, activation="linear", use_bias=False))

# Ajouter une couche de sortie avec 1 neurone (classification binaire) et la fonction d'activation personnalisée
model.add(Dense(1, activation="linear", use_bias=False))

# Compiler le modèle en spécifiant la fonction de perte, l'optimiseur et les métriques
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Résumé du modèle
model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 2)                 100       
                                                                 
 dense_16 (Dense)            (None, 1)                 2         
                                                                 
Total params: 102 (408.00 Byte)
Trainable params: 102 (408.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [81]:
NUM_EPOCHS = 100
BATCH_SIZE = 512
# Entraîner le modèle
model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

# Évaluer les performances du modèle sur les données de test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Perte (Loss) : {loss}")
print(f"Précision (Accuracy) : {accuracy}")


Epoch 1/100
1/1 [==============================] - 0s 6ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 5ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 5ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 7.6246 - accuracy: 0.5000
Epoch 11/100
1/1 [===========

In [79]:
# Obtenir les poids associés aux connexions
weights = model.get_weights()

print(len(weights))
# Afficher les poids pour chaque couche
print("Poids de la couche1 (entre la couche d'entrée et la couche cachée):")
print(weights[0])
print(len(weights[0]))

print("Poids de la couche2 (entre la couche cachée et la couche de sortie):")
print(weights[1])
print(len(weights[1]))


2
Poids de la couche1 (entre la couche d'entrée et la couche cachée):
[[-0.03114888  0.12865078]
 [-0.13958174 -0.13517712]
 [ 0.18037671  0.00269783]
 [ 0.15388513 -0.13814147]
 [-0.28742787  0.19888413]
 [-0.24686727 -0.14144476]
 [-0.177425    0.24368393]
 [-0.02762419 -0.0360243 ]
 [ 0.2658695   0.25982612]
 [-0.01722783  0.19732344]
 [ 0.27545905 -0.14818124]
 [ 0.25959116  0.30352652]
 [-0.00165358 -0.14522004]
 [-0.23167625  0.112618  ]
 [-0.07405859  0.31720412]
 [ 0.01606292  0.26189095]
 [-0.10758136 -0.21984515]
 [ 0.1301927  -0.20666891]
 [-0.13813913 -0.30108005]
 [ 0.11797827  0.00660002]
 [ 0.08499756 -0.01694989]
 [-0.32430252 -0.12297904]
 [ 0.23146987 -0.04183444]
 [-0.28426275  0.1931743 ]
 [ 0.0566377  -0.10991645]
 [ 0.04824802  0.27471107]
 [ 0.20570713 -0.19931288]
 [ 0.17706978 -0.04570487]
 [ 0.1954112  -0.02586174]
 [ 0.05255815 -0.01908785]
 [-0.17782895  0.08346221]
 [ 0.3337732  -0.02539736]
 [-0.2544166   0.12868261]
 [ 0.19843912  0.02175644]
 [-0.0108455

In [ ]:
####################################################################
#### NOUVELLES SOLUTIONS ET NOUVEAUX CLUSTERING A CHAQUE ITERATION
####################################################################

###  -------------- Iteration of the solver with the preferences
it = 1
tps = runtime
list_min_obj = [min(list_obj)]
list_min_obj_global = [min(list_obj)]

criterion = (tps < tps_max) and (it < it_max) 


# ----------------- Add the preferences to the model
while criterion :
    print("\n--------Iteration {}---------".format(it))
    it += 1

    # Create the weights of the neural network
    weights10 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons[0])]]
    weights1 = weights10 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons[i-1])] for j in range(nb_neurons[i])] for i in range(1, nb_hidden_layers + 1)]
    
    for i in range(len(weights1)):
        for j in range(len(weights1[i])):
            for k in range(len(weights1[i][j])):
                model.add(weights1[i][j][k])


    outputvar_NN1 = model.binary_var(name="outputvar_NN1")
    model.add(outputvar_NN1)


    if len(list_weights) == 2:
        weights20 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons[0])]]
        weights2 = weights20 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons[i-1])] for j in range(nb_neurons[i])] for i in range(1, nb_hidden_layers + 1)]
    
        for i in range(len(weights2)):
            for j in range(len(weights2[i])):
                for k in range(len(weights2[i][j])):
                    model.add(weights2[i][j][k])

        outputvar_NN2 = model.binary_var(name="outputvar_NN2")
        model.add(outputvar_NN2)


    # Add the constraint that the weights of the neural network must be different from the previous one
        bb = model.integer_var(0,1)
        solver.add_variable(bb)
        bb=1

        list_variables = solver.get_variables()
        variables = list_variables[0]

        b = model.integer_var(0,1)
        solver.add_variable(b)
        b=0

        for i in range(len(weights1)):
            for j in range(len(weights1[i])):
                for k in range(len(weights1[i][j])):

                    b =max(b, weights1[i][j][k] != weights2[i][j][k])

        b = (b!=0)
        bb = bb * b
        solver.add_constraint(model, bb==1)

    # Train the neural network
        for order in range(len(sol_layers)):
            for sol in sol_layers[order]:

                output_NN1 = nn.activation_function(model, solver, sol, weights1, nb_hidden_layers, nb_neurons, optimalval)
                output_NN2 = nn.activation_function(model, solver, sol, weights2, nb_hidden_layers, nb_neurons, optimalval)
                if order == 0:
                    solver.add_constraint(model, output_NN1 == 1)
                    solver.add_constraint(model, output_NN2 == 1)
                else:
                    solver.add_constraint(model, output_NN1 == 0)
                    solver.add_constraint(model, output_NN2 == 0)


    # Compare the result of 2 different neural networks for the same solution

    tasks_starts = []
    for i in range(n):
        for j in range(m):
            tasks_starts.append(model.start_of(variables[i][j]))

    solver.add_constraint(model, outputvar_NN1 == nn.activation_function(model, solver, tasks_starts, weights1, nb_hidden_layers, nb_neurons, optimalval))
    print(outputvar_NN1)

    if len(list_weights) == 2:
        solver.add_constraint(model, outputvar_NN2 == nn.activation_function(model, solver, tasks_starts, weights2, nb_hidden_layers, nb_neurons, optimalval))
        print(outputvar_NN2)
        solver.add_constraint(model, outputvar_NN1 != outputvar_NN2)

        # if it > 5:
        #     solver.add_constraint(model, outputvar_NN1 == 1)
        #     solver.add_constraint(model, outputvar_NN2 == 1)
        # else:
        #     solver.add_constraint(model, outputvar_NN1 != outputvar_NN2)
    else:
        solver.add_constraint(model, outputvar_NN1 == 1)
    

    # ------------ Solve the model
    print("\nSolving the model...")
    msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)
    print("The number of solutions generated is :",nb_solution)

    list = []
    for sol in msol:
        list.append(user.objectiveFunction(sol)+user.objectiveFunctionRegularity(sol, n, m))

    list_min_obj.append(min(list))
    print("Objective function :", list_min_obj)

    # ------------ Display the result
    fm.display_solution(msol, display_sol)
    print("Model solved !")

    # ---------------- Interaction with the user
    list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)
    print("Il y a {} solution(s)".format(len(pref)))

    sol_layers = fm.list_list_list_start_of_tasks(n, m, list_layers)

    # list_min_obj_global.append(min(list_obj))
    # print("Objective function global :", list_min_obj_global)

    # Vector of the start time of each task of each preference
    starts = user.start_pref(n, m, display_start)

    # Matrix of the start time of each task of each preference
    matrix = user.matrix_pref(n, m, display_matrix)

    # Testing the order of preferences and the differences between solutions
    fm.test(n, m, user)

#------------------ Condition d'arrêt ------------------
    tps += runtime
    criterion = (tps < tps_max) and (it < it_max)
    fm.stopCondition(it, it_max, tps, tps_max)

In [ ]:
# # Plot du minimum de l'objectif parmis les nouvelles solutions générées

# import matplotlib.pyplot as plt
# print(list_min_obj)
# plt.plot([i for i in range(it)], list_min_obj, label='min obj', marker='o')
# plt.xlabel("Iteration")
# plt.ylabel("makespan")
# plt.title("Evolution of the makespan for generate solutions")
# plt.xticks(range(it))
# plt.legend()
# plt.show() 

In [ ]:
# # Plot du minimum de l'objectif parmis les solutions générées so far

# print(list_min_obj_global)
# plt.plot([i for i in range(it-1)], list_min_obj_global, label='min obj', marker='o')
# plt.xlabel("Iteration")
# plt.ylabel("makespan")
# plt.title("Evolution of the best makespan so far")
# plt.xticks(range(it))
# plt.legend()
# plt.show() 

In [ ]:
# pref = user.getPreferences()
# print(len(pref))

# # ------------ Generate Gantt Chart
# start_times = []
# end_times = []
# task_lengths = []

# n_sol = 0
# for sol in pref:
#     n_sol = n_sol + 1
#     sol_list = sol.get_all_var_solutions()
    
#     mkspan=max([sol_list[i].get_end() for i in range(len(sol_list))] )
#     x = [ [] for i in range(m)]
#     y = [ [] for i in range(m)]
#     mp = [ {} for i in range(m)]
#     sz=len(sol_list)
#     for i in range(sz) : 
#         ind=T_machine[i]
#         x[ind].append(sol_list[i].get_start())
#         mp[ind][sol_list[i].get_start()]=ind
#         x[ind].append(sol_list[i].get_end())
#         mp[ind][sol_list[i].get_end()]=ind+1
#     eps=0.5
#     for i in range(m):
#         x[i]=sorted(x[i])
#         y[i]=[mp[i][x[i][j]]+i*eps for j in range(len(x[i]))]
#         x[i]=[0]+x[i]
#         y[i]=[i+i*eps]+y[i]
#         x[i]=x[i]+[mkspan]
#         y[i].append(i+i*eps)
#         plt.step(x[i],y[i])
#     plt.title("Gantt Chart of the solution n°" + str(n_sol))
#     plt.show()
#     print("Makespan of the solution n°" + str(n_sol) + " : " + str(mkspan))
#     print("Regularity of the solution n°" + str(n_sol) + " : " + str(user.objectiveFunctionRegularity(sol, n, m)))
#     print("Objective function of the solution n°" + str(n_sol) + " : " + str(user.objectiveFunction(sol)+user.objectiveFunctionRegularity(sol, n, m)))